In [1]:
from llama_index import VectorStoreIndex
from neo4j import GraphDatabase


In [2]:
print("Basic imports successful")

Basic imports successful


In [28]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password1234"

In [12]:
def test_connection():
  print("Connection Details:")
  print(f"URI: {uri}")
  print(f"Username: {username}")

  try:
      driver = GraphDatabase.driver(uri, auth=(username, password))
      print("\nTesting connection...")

      # Verify connectivity
      driver.verify_connectivity()
      print("✓ Connection verified!")

      # Test simple query
      with driver.session() as session:
          result = session.run("RETURN 'Connected!' as message")
          print(f"✓ Query result: {result.single()['message']}")

      return True

  except Exception as e:
      print("\n✗ Connection Error:")
      print(f"Type: {type(e).__name__}")
      print(f"Message: {str(e)}")
      return False

  finally:
      try:
          driver.close()
          print("Connection closed")
      except:
          pass

In [13]:
test_connection()

Connection Details:
URI: bolt://localhost:7687
Username: neo4j

Testing connection...
✓ Connection verified!
✓ Query result: Connected!
Connection closed


True

In [29]:
def create_test_nodes():
  driver = GraphDatabase.driver(uri, auth=(username, password))

  try:
      with driver.session() as session:
          # Create a test node
          result = session.run("""
              CREATE (n:Test {name: 'Test Node'})
              RETURN n.name as name
          """)
          print(f"Created node: {result.single()['name']}")

          # Count all nodes
          result = session.run("MATCH (n) RETURN count(n) as count")
          print(f"Total nodes in database: {result.single()['count']}")

  finally:
      driver.close()


In [30]:
# Test creating nodes
create_test_nodes()

Created node: Test Node
Total nodes in database: 9


In [31]:
def create_healthcare_schema():
  driver = GraphDatabase.driver(uri, auth=(username, password))

  try:
      with driver.session() as session:
          # First, let's clear existing data
          session.run("MATCH (n) DETACH DELETE n")

          # Create constraints for unique IDs
          session.run("CREATE CONSTRAINT patient_id IF NOT EXISTS FOR (p:Patient) REQUIRE p.id IS UNIQUE")
          session.run("CREATE CONSTRAINT doctor_id IF NOT EXISTS FOR (d:Doctor) REQUIRE d.id IS UNIQUE")

          # Create some sample data
          session.run("""
              // Create Doctors
              CREATE (d1:Doctor {id: 'D001', name: 'Dr. Smith', specialty: 'Cardiology'})
              CREATE (d2:Doctor {id: 'D002', name: 'Dr. Jones', specialty: 'Neurology'})

              // Create Patients
              CREATE (p1:Patient {id: 'P001', name: 'John Doe', age: 45, gender: 'M'})
              CREATE (p2:Patient {id: 'P002', name: 'Jane Smith', age: 32, gender: 'F'})

              // Create Conditions
              CREATE (c1:Condition {name: 'Hypertension', icd10: 'I10'})
              CREATE (c2:Condition {name: 'Diabetes', icd10: 'E11'})

              // Create Medications
              CREATE (m1:Medication {name: 'Lisinopril', dosage: '10mg'})
              CREATE (m2:Medication {name: 'Metformin', dosage: '500mg'})

              // Create Relationships
              CREATE (p1)-[:DIAGNOSED_WITH {date: '2023-01-15'}]->(c1)
              CREATE (p1)-[:PRESCRIBED {date: '2023-01-15'}]->(m1)
              CREATE (d1)-[:TREATS]->(p1)
              CREATE (p2)-[:DIAGNOSED_WITH {date: '2023-02-20'}]->(c2)
              CREATE (p2)-[:PRESCRIBED {date: '2023-02-20'}]->(m2)
              CREATE (d2)-[:TREATS]->(p2)
          """)

          print("Healthcare schema and sample data created successfully!")

  finally:
      driver.close()

def perform_sample_queries():
  driver = GraphDatabase.driver(uri, auth=(username, password))

  try:
      with driver.session() as session:
          # Query 1: Find all patients and their conditions
          print("\n1. Patients and their conditions:")
          result = session.run("""
              MATCH (p:Patient)-[r:DIAGNOSED_WITH]->(c:Condition)
              RETURN p.name as Patient, c.name as Condition, r.date as DiagnosisDate
          """)
          for record in result:
              print(f"Patient: {record['Patient']} - Condition: {record['Condition']} (Diagnosed: {record['DiagnosisDate']})")

          # Query 2: Find doctors and their patients
          print("\n2. Doctors and their patients:")
          result = session.run("""
              MATCH (d:Doctor)-[:TREATS]->(p:Patient)
              RETURN d.name as Doctor, d.specialty as Specialty, collect(p.name) as Patients
          """)
          for record in result:
              print(f"Doctor: {record['Doctor']} ({record['Specialty']}) - Patients: {record['Patients']}")

          # Query 3: Medications prescribed to patients
          print("\n3. Patient medications:")
          result = session.run("""
              MATCH (p:Patient)-[:PRESCRIBED]->(m:Medication)
              RETURN p.name as Patient, m.name as Medication, m.dosage as Dosage
          """)
          for record in result:
              print(f"Patient: {record['Patient']} - Medication: {record['Medication']} ({record['Dosage']})")

  finally:
      driver.close()


In [32]:
create_healthcare_schema()

Healthcare schema and sample data created successfully!


In [33]:
perform_sample_queries()


1. Patients and their conditions:
Patient: John Doe - Condition: Hypertension (Diagnosed: 2023-01-15)
Patient: Jane Smith - Condition: Diabetes (Diagnosed: 2023-02-20)

2. Doctors and their patients:
Doctor: Dr. Smith (Cardiology) - Patients: ['John Doe']
Doctor: Dr. Jones (Neurology) - Patients: ['Jane Smith']

3. Patient medications:
Patient: John Doe - Medication: Lisinopril (10mg)
Patient: Jane Smith - Medication: Metformin (500mg)


In [34]:
def healthcare_qa(question):
  driver = GraphDatabase.driver(uri, auth=(username, password))

In [35]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password1234"  # Replace with your actual password

def test_connection():
  try:
      driver = GraphDatabase.driver(uri, auth=(username, password))
      with driver.session() as session:
          result = session.run("RETURN 1")
          print("Connection successful:", result.single()[0] == 1)
  except Exception as e:
      print("Connection failed:", e)
  finally:
      driver.close()

test_connection()

Connection successful: True


In [36]:
from neo4j import GraphDatabase

# Connection details matching your Neo4j Desktop configuration
uri = "bolt://localhost:7687"    # Using bolt protocol on default port 7687
username = "neo4j"               # Default Neo4j username
password = "password1234"   # The password you set

def healthcare_qa(question):
  """
  Function to handle natural language questions about healthcare data
  Parameters:
      question (str): Natural language question from user
  Returns:
      str: Answer to the question based on database query
  """
  # Create a driver instance to connect to Neo4j
  driver = GraphDatabase.driver(uri, auth=(username, password))

  # Convert question to lowercase to make matching easier
  question = question.lower()

  try:
      # Create a session for database operations
      with driver.session() as session:

          # Pattern 1: Count total patients
          if "how many patients" in question:
              # Simple query to count Patient nodes
              result = session.run("""
                  MATCH (p:Patient) 
                  RETURN count(p) as count
              """)
              count = result.single()["count"]
              return f"There are {count} patients in the database."

          # Pattern 2: Patient conditions query
          elif "what conditions" in question and "patient" in question:
              # Query to find conditions for all patients
              result = session.run("""
                  MATCH (p:Patient)-[:DIAGNOSED_WITH]->(c:Condition)
                  RETURN 
                      p.name as patient,
                      collect(c.name) as conditions
              """)
              # Format the response
              response = ""
              for record in result:
                  conditions_list = ', '.join(record['conditions'])
                  response += f"{record['patient']} has the following conditions: {conditions_list}\n"
              return response

          # Pattern 3: Doctor-Patient relationships
          elif "who treats" in question or "doctor for" in question:
              # Query to find which doctors treat which patients
              result = session.run("""
                  MATCH (d:Doctor)-[:TREATS]->(p:Patient)
                  RETURN 
                      d.name as doctor,
                      d.specialty as specialty,
                      p.name as patient
              """)
              # Format the response
              response = ""
              for record in result:
                  response += f"Dr. {record['doctor']} ({record['specialty']}) treats {record['patient']}\n"
              return response

          # Pattern 4: Medication information
          elif "medications" in question:
              # Query to find medications prescribed to patients
              result = session.run("""
                  MATCH (p:Patient)-[:PRESCRIBED]->(m:Medication)
                  RETURN 
                      p.name as patient,
                      collect({
                          name: m.name,
                          dosage: m.dosage
                      }) as medications
              """)
              # Format the response
              response = ""
              for record in result:
                  # Create a formatted list of medications with dosages
                  meds = [f"{m['name']} ({m['dosage']})" for m in record['medications']]
                  response += f"{record['patient']} is prescribed: {', '.join(meds)}\n"
              return response

          # Pattern 5: Doctors by specialty
          elif "doctors" in question and "specialty" in question:
              # Query to group doctors by their specialties
              result = session.run("""
                  MATCH (d:Doctor)
                  RETURN 
                      d.specialty as specialty,
                      collect(d.name) as doctors
              """)
              # Format the response
              response = "Doctors by specialty:\n"
              for record in result:
                  response += f"{record['specialty']}: {', '.join(record['doctors'])}\n"
              return response

          # Default response for unrecognized questions
          else:
              return """I'm sorry, I don't understand that question. Try asking about:
              - Number of patients
              - Patient conditions
              - Doctor-patient relationships
              - Medications prescribed
              - Doctors by specialty"""

  finally:
      # Always close the driver connection
      driver.close()

# Example usage with test questions
def test_qa_system():
  test_questions = [
      "How many patients are in the database?",
      "What conditions does John Doe have?",
      "Who treats Jane Smith?",
      "What medications are prescribed to patients?",
      "Show me all doctors by specialty"
  ]

  print("Healthcare Knowledge Graph QA System Test")
  print("-" * 50)
  for question in test_questions:
      print(f"\nQuestion: {question}")
      print(f"Answer: {healthcare_qa(question)}")

# Run the test
test_qa_system()

Healthcare Knowledge Graph QA System Test
--------------------------------------------------

Question: How many patients are in the database?
Answer: There are 2 patients in the database.

Question: What conditions does John Doe have?
Answer: I'm sorry, I don't understand that question. Try asking about:
              - Number of patients
              - Patient conditions
              - Doctor-patient relationships
              - Medications prescribed
              - Doctors by specialty

Question: Who treats Jane Smith?
Answer: Dr. Dr. Smith (Cardiology) treats John Doe
Dr. Dr. Jones (Neurology) treats Jane Smith


Question: What medications are prescribed to patients?
Answer: John Doe is prescribed: Lisinopril (10mg)
Jane Smith is prescribed: Metformin (500mg)


Question: Show me all doctors by specialty
Answer: Doctors by specialty:
Cardiology: Dr. Smith
Neurology: Dr. Jones

